<a href="https://colab.research.google.com/github/c-quilo/premiereDroplets/blob/main/ganDropletExperiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Generative adversarial Networks (GANs) for droplet experiments

In [15]:
from __future__ import print_function, division

import tensorflow.keras as tf
import tensorflow
from sklearn.model_selection import train_test_split
import tensorflow.keras.backend as backend
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from keras import backend
from keras.layers import Lambda
from keras.constraints import Constraint
from keras.initializers import RandomNormal

from keras import optimizers
from keras.utils import np_utils
import tensorflow.keras as tf

In [16]:
sns.__version__

'0.11.1'

The new class GANexperiments is defined

In [23]:
class ClipConstraint(Constraint):
    # set clip value when initialized
    def __init__(self, clip_value):
        self.clip_value = clip_value

    # clip model weights to hypercube
    def __call__(self, weights):
        return backend.clip(weights, -self.clip_value, self.clip_value)

    # get the config
    def get_config(self):
        return {'clip_value': self.clip_value}

# clip model weights to a given hypercube
class dropletGAN():

    def __init__(self, directory, trainingData, latent_dim, GANorWGAN):

        # Wasserstein loss
        def wasserstein_loss(y_true, y_pred):
            return backend.mean(y_true * y_pred)

        self.directory = directory
        self.trainingData = trainingData
        self.nFeatures = self.trainingData.shape[1]
        # Dimension of the latent space (noise)
        self.latent_dim = latent_dim
        self.constraint = 0.01
        self.dropoutNumber = 0.5
        self.alpha = 0.3
        self.GANorWGAN = GANorWGAN
        self.nameExperiment = '_dropletExperiments_'

        self.c1_hist = []
        self.c2_hist = []
        self.g_hist = []

        self.optimizer = tf.optimizers.Nadam()

        if self.GANorWGAN == 'WGAN':
            self.loss = wasserstein_loss
        elif self.GANorWGAN == 'GAN':
            self.loss = 'binary_crossentropy'

        self.loss_gen = 'mse'

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()

        # Build the encoder and decoder
        self.generator = self.build_generator()

        # Only the generator is trained through the combined model, thus:
        self.discriminator.trainable = False

        # Connecting models
        noise_input = tf.Input(shape=self.latent_dim)
        generator_output = self.generator(noise_input)
        discriminator_output = self.discriminator(generator_output)

        # The combined model stacks the autoencoder and discriminator
        # The stacked model has one input and two outputs: the decoded input and the discriminator output
        self.combined = tf.Model(noise_input, discriminator_output, name = 'dropletGAN')
        self.combined.compile(loss=self.loss, loss_weights=[0.999, 0.001], optimizer=self.optimizer)

    def build_discriminator(self):
        init = RandomNormal(stddev=0.02)
        const = ClipConstraint(0.01)

        in_disc = tf.Input(shape=(self.nFeatures))
        disc = tf.layers.LeakyReLU(self.alpha)(in_disc)
        disc = tf.layers.BatchNormalization()(disc)
        disc_output = tf.layers.Dense(1, activation='sigmoid')(disc)
        discriminator = tf.Model(in_disc, disc_output, name='Discriminator')
        discriminator.compile(loss=self.loss, optimizer=self.optimizer)

        return discriminator

    def build_generator(self):
        #init = RandomNormal(stddev=0.02)
        #init = tf.initializers.RandomNormal(stddev=0.02)

        input_gen = tf.Input(shape=self.latent_dim)
        gen = tf.layers.Dense(8)(input_gen)
        gen = tf.layers.LeakyReLU(self.alpha)(gen)
        gen = tf.layers.BatchNormalization()(gen)
        gen = tf.layers.Dense(8)(gen)
        gen = tf.layers.LeakyReLU(self.alpha)(gen)
        gen = tf.layers.BatchNormalization()(gen)
        gen_output = tf.layers.Dense(self.nFeatures, activation='tanh')(gen)

        generator = tf.Model(input_gen, gen_output, name='Generator')
        generator.summary()
        return generator

    def train(self, epochs, batch_size=128, sample_interval=50, n_critic=5):

        # Load and pre process the data
        global X_all
        X_all = self.trainingData

        def scaler(x, xmin, xmax, min, max):
          scale = (max - min) / (xmax - xmin)
          xScaled = scale * x + min - xmin * scale
          return xScaled
        
        minData = np.min(X_all)
        maxData = np.max(X_all)

        X_all = scaler(X_all, minData, maxData, -1, 1)

        if self.GANorWGAN == 'WGAN':
            real = -np.ones(batch_size)
            fake = np.ones(batch_size)

        if self.GANorWGAN == 'GAN':
            real = np.ones(batch_size)
            fake = np.zeros(batch_size)

        # Training the model
        for epoch in range(epochs):
            c1_tmp, c2_tmp = list(), list()

            # Training the discriminator more often than the generator
            for _ in range(n_critic):
                # Randomly selected samples and noise
                randomIndex = np.random.randint(0, X_all.shape[0], size=batch_size)
                noise = np.random.normal(0, 1, size=(batch_size, self.latent_dim))
                # Select a random batch of input
                real_seqs = X_all[randomIndex]

                # Generate a batch of new outputs (in the latent space) predicted by the encoder
                gen_seqs = self.generator.predict(noise)

                # Train the discriminator
                # The arbitrary noise is considered to be a "real" sample
                d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
                c1_tmp.append(d_loss_real)
                # The latent space generated by the encoder is considered a "fake" sample
                d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
                c2_tmp.append(d_loss_fake)

            self.c1_hist.append(np.mean(c1_tmp))
            self.c2_hist.append(np.mean(c2_tmp))

            # Training the stacked model
            g_loss = self.combined.train_on_batch(noise, real)
            self.g_hist.append(g_loss)
            print("%d [C1 real: %f, C2 fake: %f], [G loss: %f]" % (epoch, self.c1_hist[epoch], self.c2_hist[epoch], g_loss))

            # Checkpoint progress: Plot losses and predicted data
            if epoch % sample_interval == 0:

                self.plot_loss(epoch)
                self.plot_values(epoch)
                self.generator.save(self.directory + '/' + self.nameExperiment + GANorWGAN +
                                            '_' + str(self.latent_dim) + '_' + str(epoch),
                                            save_format='tf')

                self.discriminator.save(self.directory + '/' + self.nameExperiment + GANorWGAN + 
                                        '_' + str(self.latent_dim) + '_' + str(epoch),
                                        save_format='tf')

    # Plots the (W)GAN related losses at every sample interval

    def plot_loss(self, epoch):
        fig = plt.figure()
        plt.plot(self.c1_hist, c='red')
        plt.plot(self.c2_hist, c='blue')
        plt.plot(self.g_hist, c='green')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title("GAN Loss per Epoch")
        plt.legend(['C real', 'C fake', 'Generator'])

        plt.savefig(self.directory + '/' + self.nameExperiment + GANorWGAN + '_losses_'  + str(epoch) +
                    '_' + str(self.latent_dim) + '.png')
        plt.close()

    # Plots predicted in the first 8 latent dimension at every sample interval

    def plot_values(self, epoch):
        fig = plt.figure(0, figsize=(20,10))
        noise = np.random.normal(0, 1, size=(X_all.shape[0], self.latent_dim))
        prediction = self.generator(noise)
        nSamples = X_all.shape[0]
        sns.distplot(X_all[:])
        sns.distplot(prediction[:])

        plt.legend(['Ground truth', 'Generated data'])
        
        plt.tight_layout()
        plt.savefig(self.directory + '/' + self.nameExperiment + GANorWGAN + '_generated_data_' + str(epoch) +
                    '_' + str(self.latent_dim) + '.png')
        plt.close()

Execute the class and train the GAN

In [ ]:
if __name__ == '__main__':
    #Load data
    directory = '/content/'
    filename = 'normalisedDropletsExperiments.npy'
    trainingData = np.load(directory + filename)
    epochs = 10000
    batch_size = 32
    n_critic = 5
    sample_interval = 1000
    latent_dim = 4

    #Training method
    GANorWGAN = 'GAN'

    dropGAN = dropletGAN(directory = directory,
              trainingData=trainingData,
              latent_dim=latent_dim,
              GANorWGAN=GANorWGAN)
    dropGAN.train(epochs=epochs,
              batch_size=batch_size,
              sample_interval=sample_interval,
              n_critic = n_critic)

Model: "Generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        [(None, 4)]               0         
_________________________________________________________________
dense_41 (Dense)             (None, 8)                 40        
_________________________________________________________________
leaky_re_lu_31 (LeakyReLU)   (None, 8)                 0         
_________________________________________________________________
batch_normalization_31 (Batc (None, 8)                 32        
_________________________________________________________________
dense_42 (Dense)             (None, 8)                 72        
_________________________________________________________________
leaky_re_lu_32 (LeakyReLU)   (None, 8)                 0         
_________________________________________________________________
batch_normalization_32 (Batc (None, 8)                 32

/usr/local/lib/python3.7/dist-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.7/dist-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


INFO:tensorflow:Assets written to: /content//_dropletExperiments_GAN_4_0/assets
INFO:tensorflow:Assets written to: /content//_dropletExperiments_GAN_4_0/assets
1 [C1 real: 0.757576, C2 fake: 0.872432], [G loss: 0.760674]
2 [C1 real: 0.751540, C2 fake: 0.817146], [G loss: 0.738099]
3 [C1 real: 0.751215, C2 fake: 0.837008], [G loss: 0.761219]
4 [C1 real: 0.742439, C2 fake: 0.802033], [G loss: 0.762737]
5 [C1 real: 0.739764, C2 fake: 0.806056], [G loss: 0.720517]
6 [C1 real: 0.737399, C2 fake: 0.813249], [G loss: 0.738977]
7 [C1 real: 0.740117, C2 fake: 0.782373], [G loss: 0.714047]
8 [C1 real: 0.733970, C2 fake: 0.779034], [G loss: 0.726330]
9 [C1 real: 0.735335, C2 fake: 0.762449], [G loss: 0.734232]
10 [C1 real: 0.733797, C2 fake: 0.753447], [G loss: 0.700430]
11 [C1 real: 0.724324, C2 fake: 0.733002], [G loss: 0.714770]
12 [C1 real: 0.738874, C2 fake: 0.743072], [G loss: 0.674033]
13 [C1 real: 0.730886, C2 fake: 0.739876], [G loss: 0.685650]
14 [C1 real: 0.726041, C2 fake: 0.735135], 

/usr/local/lib/python3.7/dist-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.7/dist-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


INFO:tensorflow:Assets written to: /content//_dropletExperiments_GAN_4_1000/assets
INFO:tensorflow:Assets written to: /content//_dropletExperiments_GAN_4_1000/assets
1001 [C1 real: 0.693205, C2 fake: 0.693198], [G loss: 0.692505]
1002 [C1 real: 0.693205, C2 fake: 0.693198], [G loss: 0.692502]
1003 [C1 real: 0.693205, C2 fake: 0.693198], [G loss: 0.692506]
1004 [C1 real: 0.693205, C2 fake: 0.693198], [G loss: 0.692504]
1005 [C1 real: 0.693205, C2 fake: 0.693198], [G loss: 0.692505]
1006 [C1 real: 0.693205, C2 fake: 0.693198], [G loss: 0.692505]
1007 [C1 real: 0.693205, C2 fake: 0.693198], [G loss: 0.692503]
1008 [C1 real: 0.693205, C2 fake: 0.693198], [G loss: 0.692498]
1009 [C1 real: 0.693205, C2 fake: 0.693198], [G loss: 0.692497]
1010 [C1 real: 0.693205, C2 fake: 0.693198], [G loss: 0.692497]
1011 [C1 real: 0.693205, C2 fake: 0.693198], [G loss: 0.692496]
1012 [C1 real: 0.693204, C2 fake: 0.693198], [G loss: 0.692497]
1013 [C1 real: 0.693204, C2 fake: 0.693198], [G loss: 0.692499]
10

/usr/local/lib/python3.7/dist-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.7/dist-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


INFO:tensorflow:Assets written to: /content//_dropletExperiments_GAN_4_2000/assets
INFO:tensorflow:Assets written to: /content//_dropletExperiments_GAN_4_2000/assets
2001 [C1 real: 0.693188, C2 fake: 0.693187], [G loss: 0.692430]
2002 [C1 real: 0.693187, C2 fake: 0.693187], [G loss: 0.692632]
2003 [C1 real: 0.693187, C2 fake: 0.693187], [G loss: 0.692432]
2004 [C1 real: 0.693187, C2 fake: 0.693187], [G loss: 0.692504]
2005 [C1 real: 0.693187, C2 fake: 0.693187], [G loss: 0.692476]
2006 [C1 real: 0.693187, C2 fake: 0.693187], [G loss: 0.692536]
2007 [C1 real: 0.693187, C2 fake: 0.693187], [G loss: 0.692352]
2008 [C1 real: 0.693187, C2 fake: 0.693187], [G loss: 0.692974]
2009 [C1 real: 0.693187, C2 fake: 0.693187], [G loss: 0.691819]
2010 [C1 real: 0.693187, C2 fake: 0.693187], [G loss: 0.692626]
2011 [C1 real: 0.693187, C2 fake: 0.693187], [G loss: 0.692478]
2012 [C1 real: 0.693187, C2 fake: 0.693187], [G loss: 0.692500]
2013 [C1 real: 0.693187, C2 fake: 0.693187], [G loss: 0.692485]
20